In [1]:
import copy
import random
import os
import time

from IPython.display import clear_output, display, HTML

from collections import Counter
from PIL import Image

import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

sns.set(style="darkgrid")

import tensorflow as tf

# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()
# tf.debugging.set_log_device_placement(True)
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    # tf.config.experimental.set_memory_growth(gpu, True)
    # tf.config.set_visible_devices(gpu, 'GPU')
    tf.config.set_logical_device_configuration(gpu, [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
# os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(
#     str(i) for i in range(len(tf.config.experimental.list_logical_devices('GPU'))))
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

strategy = tf.distribute.MirroredStrategy(tf.config.list_logical_devices('GPU'),
                                          cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

keras = tf.keras
from keras.utils import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T

2022-11-26 09:12:30.410777: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 09:12:30.515096: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-26 09:12:30.913793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sudha/cuda/lib64:/usr/local/cuda-11.2/lib64:/home/sudha/miniconda3/envs/tf/lib/
2022-11-26 09:12:30.913843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-11-26 09:12:31.355029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 09:12:31.355224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 09:12:31.360053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 09:12:31.360246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 09:12:31.360371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [2]:
# Constants
BASE_PATH = "../src/data"
DATA_PATH = f"{BASE_PATH}/external"
STATE_PATH = f"{BASE_PATH}/state"
IMAGE_PATH = f'{DATA_PATH}/ISIC_2019_Training_Input'
IMAGE_TYPE = 'jpg'
MODELS_PATH = f"{BASE_PATH}/models"
TARGET_SIZE = (224, 224)
CHANNELS = 3
CONTRAST_FACTOR = 3  # 2 is performing better and so choosing that instead
# CONTRAST_FACTOR = 2
DELTA = 0.3
IS_FLIP_ON = True
RANDOM_STATE = 42
# KERNEL_SIZE = (5, 5)  # decreasing seeing 3,3 doing better
KERNEL_SIZE = (3, 3)
# LEARNING_RATE = 0.001  # decreasing seeing too much ripple effect
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
GLOBAL_BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync
print(f"Global Batch Size: {GLOBAL_BATCH_SIZE}")
TEST_SIZE = 0.2
# EPOCHS = 5  # validation accuracy seems to increase with the new learning rate and so increasing the number of epochs
EPOCHS = 25

Global Batch Size: 64


In [3]:
def reset_random_seeds(seed=RANDOM_STATE):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


def clear_session():
    tf.keras.backend.clear_session()

In [4]:
df_metadata = pd.read_csv(f'{DATA_PATH}/ISIC_2019_Training_Metadata.csv')
df_truth = pd.read_csv(f'{DATA_PATH}/ISIC_2019_Training_GroundTruth.csv')

In [5]:
labels = df_truth.columns
labels = labels[1:]
print(labels)
label_mapping = {i: label for i, label in enumerate(labels)}
print(label_mapping)

Index(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], dtype='object')
{0: 'MEL', 1: 'NV', 2: 'BCC', 3: 'AK', 4: 'BKL', 5: 'DF', 6: 'VASC', 7: 'SCC', 8: 'UNK'}


In [6]:
dense_labels = df_truth[labels]
dense_labels = dense_labels.values
print('Dense Labels...')
display(dense_labels)
truth_labels = np.argmax(dense_labels, axis=-1)
print('Truth Labels...')
display(truth_labels)

Dense Labels...


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Truth Labels...


array([1, 1, 0, ..., 0, 1, 4])

In [7]:
df_truth['label'] = truth_labels.tolist()
display(df_truth)

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [8]:
df_joined = df_metadata.join(df_truth.set_index('image'), on='image', how='left')
display(df_joined)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [9]:
# Site details add more information to model and so fill with unknown if not available
print(
    f"Count of null values in anatom_site_general column before null fill: {sum(df_joined['anatom_site_general'].isnull())}")
df_joined['anatom_site_general'] = df_joined['anatom_site_general'].fillna('unknown')
print(
    f"Count of null values in anatom_site_general column after null fill: {sum(df_joined['anatom_site_general'].isnull())}")

Count of null values in anatom_site_general column before null fill: 2631
Count of null values in anatom_site_general column after null fill: 0


In [10]:
# Sex details add more information to model and so fill with unknown if not available
print(f"Count of null values in sex column before null fill: {sum(df_joined['sex'].isnull())}")
df_joined['sex'] = df_joined['sex'].fillna('unknown')
print(f"Count of null values in sex column after null fill: {sum(df_joined['sex'].isnull())}")

Count of null values in sex column before null fill: 384
Count of null values in sex column after null fill: 0


In [11]:
# Age is important for the model and so we remove rows with na/inf/zero values
df_cleaned = df_joined[~df_joined['age_approx'].isnull()]
print(f"Observations in cleaned dataframe before na/inf/zero remove: {len(df_cleaned)}")
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)
df_cleaned.dropna(how="all", inplace=True)
df_cleaned = df_cleaned[df_cleaned['age_approx'] > 0]
print(f"Observations in cleaned dataframe after na/inf/zero remove: {len(df_cleaned)}")

Observations in cleaned dataframe before na/inf/zero remove: 24894
Observations in cleaned dataframe after na/inf/zero remove: 24840


In [12]:
# Convert age column to uint8 for resampling
# df_cleaned['age_approx'] = df_cleaned['age_approx'].astype(np.uint8)
# display(df_cleaned)

In [13]:
# decreasing the scope of anatomy site can provide more information and so getting a more generic information
df_cleaned['anatomy_site'] = df_cleaned['anatom_site_general'].apply(lambda anatomy: anatomy.split()[-1])
display(df_cleaned)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label,anatomy_site
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,extremity
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,torso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,head/neck
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,torso
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,palms/soles


In [14]:
with open(f"{STATE_PATH}/images_seen.txt") as seen_file:
    images_seen = seen_file.read()
    images_seen = images_seen.split('\n')
    images_seen = set(images_seen)
df_cleaned['is_seen'] = df_cleaned['image'].apply(lambda name: name in images_seen)

df_seen = df_cleaned[df_cleaned['is_seen']].copy()
display(df_seen)
df_test = df_cleaned[df_cleaned['is_seen'] == False].copy()
# Limit the number of test results to speed up experiments
# df_test = df_test.head(df_seen.shape[0])
display(df_test)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label,anatomy_site,is_seen
2915,ISIC_0024318,65.0,lower extremity,HAM_0002450,female,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,extremity,True
2967,ISIC_0024370,55.0,unknown,HAM_0001780,male,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,unknown,True
2983,ISIC_0024386,40.0,lower extremity,HAM_0005112,female,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,extremity,True
3072,ISIC_0024475,35.0,head/neck,HAM_0003873,male,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,head/neck,True
3114,ISIC_0024517,65.0,posterior torso,HAM_0001894,male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7,torso,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25256,ISIC_0073141,45.0,anterior torso,BCN_0005520,female,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,torso,True
25259,ISIC_0073144,75.0,anterior torso,BCN_0004091,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,True
25287,ISIC_0073193,35.0,anterior torso,BCN_0002147,male,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,torso,True
25289,ISIC_0073195,70.0,lower extremity,BCN_0005492,male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7,extremity,True


,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label,anatomy_site,is_seen
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,False
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,False
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity,False
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,extremity,False
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,torso,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,head/neck,False
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,torso,False
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity,False
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,palms/soles,False


## Baseline

Predicting everything as MEL (label = 0) since that is the most dangerous type of skin lesion!!!

In [15]:
accuracy_test = (df_test['label'] == 0).sum() / df_test.shape[0]
print(f"Accuracy on test data: {accuracy_test * 100:.2f}%")

Accuracy on test data: 18.21%


In [16]:
def load_img_data(img_names):
    images = []
    for img_name in img_names:
        img = load_img(f"{IMAGE_PATH}/{img_name}.{IMAGE_TYPE}", target_size=TARGET_SIZE)
        img = img_to_array(img)
        images.append(img)
    return np.stack(images)

In [17]:
def aug_img_data(images, size=TARGET_SIZE, delta=DELTA, contrast_factor=CONTRAST_FACTOR, is_flip_on=IS_FLIP_ON,
                 **kwargs):
    # make a copy of images
    images_aug = copy.deepcopy(images)
    # image resize
    images_aug = tf.image.resize(images_aug, size=size)

    # brightness
    images_aug = tf.image.adjust_brightness(images_aug, delta=delta)
    # contrast
    images_aug = tf.image.adjust_contrast(images_aug, contrast_factor=contrast_factor)
    # random flip
    if is_flip_on:
        images_aug = tf.image.random_flip_left_right(images_aug)

    return images_aug

In [18]:
LIMIT = GLOBAL_BATCH_SIZE * strategy.num_replicas_in_sync
df_seen = df_seen.iloc[:LIMIT]
df_test = df_test.iloc[:LIMIT * strategy.num_replicas_in_sync]

img_seen = load_img_data(df_seen['image'])
y_seen = df_seen['label']

img_train, img_val, y_train, y_val = train_test_split(img_seen, y_seen, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print(f"Length of training data: {img_train.shape[0]}")
print(f"Length of validation data: {img_val.shape[0]}")

img_test = load_img_data(df_test['image'])
y_test = df_test['label']
print(f"Length of test data: {img_test.shape[0]}")

image_index = {name: i for i, name in enumerate(df_cleaned['image'])}
rev_image_index = {i: name for i, name in enumerate(df_cleaned['image'])}
df_seen['image'] = df_seen['image'].apply(lambda name: image_index[name])
df_test['image'] = df_test['image'].apply(lambda name: image_index[name])

Length of training data: 102
Length of validation data: 26
Length of test data: 256


In [19]:
def plot_history(hist):
    x_arr = np.arange(len(hist['loss'])) + 1

    fig = plt.figure(figsize=(12, 4))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
    ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
    ax.legend(fontsize=15)
    ax.set_xlabel('Epoch', size=15)
    ax.set_ylabel('Loss', size=15)

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
    ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
    ax.legend(fontsize=15)
    ax.set_xlabel('Epoch', size=15)
    ax.set_ylabel('Accuracy', size=15)
    plt.show()

## CNN

In [20]:
def build_cnn_model(kernel_size=KERNEL_SIZE, strides=(1, 1), pool_size=(2, 2), optimizer='Adam',
                    learning_rate=LEARNING_RATE, conv_filters=[32, 64], hidden_units=[1024, 256], dropout_rate=0.8,
                    **kwargs):
    # def build_cnn_model(kernel_size=(5, 5), strides=(1, 1), pool_size=(2, 2), optimizer='Adam', learning_rate=0.001, conv_filters=[32, 64], hidden_units=[1024], dropout_rate=0.5, **kwargs):
    clear_session()
    reset_random_seeds()

    model = keras.Sequential()

    for i, filters in enumerate(conv_filters):
        i += 1
        model.add(keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same',
                                      name=f'conv_{i}', activation='relu'))
        model.add(keras.layers.MaxPool2D(pool_size=pool_size, name=f'pool_{i}'))

    model.add(keras.layers.Flatten())

    for i, units in enumerate(hidden_units):
        i += 1
        model.add(keras.layers.Dropout(rate=dropout_rate))
        model.add(keras.layers.Dense(units=units, name=f'fc_{i}', activation='relu'))

    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=len(labels), name='output', activation="softmax"))
    model.build(input_shape=(None, *TARGET_SIZE, 3))

    optimizer_mapping = {
        'sgd': keras.optimizers.SGD,
        'adam': keras.optimizers.Adam,
    }
    optimizer = optimizer_mapping[optimizer.lower()]
    optimizer = optimizer(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

# augment train data
img_seen_aug = aug_img_data(img_seen)
img_seen_aug = tf.concat([img_seen, img_seen_aug], axis=0)
y_seen_aug = tf.concat([y_seen, y_seen], axis=0)

# shuffle the data
reset_random_seeds()
shuffle = tf.random.shuffle(tf.range(tf.shape(img_seen_aug)[0], dtype=tf.int32))
img_seen_aug = tf.gather(img_seen_aug, shuffle)
y_seen_aug = tf.gather(y_seen_aug, shuffle).numpy()
print(f"Length of augmented seen data: {img_seen_aug.shape[0]}")

if 'model_cnn' in locals() or 'model_cnn' in globals():
    del model_cnn
model_cnn = build_cnn_model()
model_cnn.summary()

history_cnn = model_cnn.fit(img_seen_aug, y_seen_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=TEST_SIZE).history
plot_history(history_cnn)

i = 1100
print(f"embeddings: {build_embedding_df(model_cnn, img_seen_aug[i:i + 1]).to_numpy()}")
print(f"predicts: {model_cnn.predict(img_seen_aug[i:i+1])}")
print(f"ground-truth: {y_seen_aug[i:i+1]}")

default_args = {
    "kernel_size": KERNEL_SIZE,
    "strides": (1, 1),
    "pool_size": (2, 2),
    "optimizer": "Adam",
    "learning_rate": LEARNING_RATE,
    "delta": DELTA,
    "contrast_factor": CONTRAST_FACTOR,
    "is_flip_on": IS_FLIP_ON
}
new_args = {
    # "kernel_size": (3, 3),
    "strides": (2, 2),
    # "pool_size": (3, 3),
    # "optimizer": "SGD",  # unable to train and so not considering this
    # "learning_rate": 0.01,  # this is causing ripple effect
    "delta": 0.1,  # this is performing better and so using as default
    "contrast_factor": 2,  # this is performing better and so using as default
    "is_flip_on": False
}
# history_cnn = history_cnn.history
results = [[
    history_cnn['accuracy'][-1],
    history_cnn['val_accuracy'][-1],
    default_args['kernel_size'],
    default_args['strides'],
    default_args['pool_size'],
    default_args['learning_rate'],
    default_args['optimizer'],
    default_args['delta'],
    default_args['contrast_factor'],
    default_args['is_flip_on']
]]

for k, v in new_args.items():
    time.sleep(30)
    # clear_output(wait=True)
    exp_args = copy.deepcopy(default_args)
    exp_args[k] = v
    print(f"Experiment: {exp_args}")
    print(f"Modified: {k} = {v}")

    img_seen_aug = aug_img_data(img_seen, **exp_args)
    img_seen_aug = tf.concat([img_seen, img_seen_aug], axis=0)
    y_seen_aug = tf.concat([y_seen, y_seen], axis=0)

    # shuffle the data
    reset_random_seeds()
    shuffle = tf.random.shuffle(tf.range(tf.shape(img_seen_aug)[0], dtype=tf.int32))
    img_seen_aug = tf.gather(img_seen_aug, shuffle)
    y_seen_aug = tf.gather(y_seen_aug, shuffle).numpy()
    # print(f"Length of augmented seen data: {img_seen_aug.shape[0]}")

    if 'model_cnn' in locals() or 'model_cnn' in globals():
        del model_cnn
    model_cnn = build_cnn_model(**exp_args)
    model_cnn.summary()

    history_cnn = model_cnn.fit(img_seen_aug, y_seen_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=TEST_SIZE).history
    plot_history(history_cnn)

    results.append([
        history_cnn['accuracy'][-1],
        history_cnn['val_accuracy'][-1],
        exp_args['kernel_size'],
        exp_args['strides'],
        exp_args['pool_size'],
        exp_args['learning_rate'],
        exp_args['optimizer'],
        exp_args['delta'],
        exp_args['contrast_factor'],
        exp_args['is_flip_on']
    ])

    i = 1100
    print(f"embeddings: {build_embedding_df(model_cnn, img_seen_aug[i:i + 1]).to_numpy()}")
    print(f"predicts: {model_cnn.predict(img_seen_aug[i:i+1])}")
    print(f"ground-truth: {y_seen_aug[i:i+1]}")

df_cnn_results = pd.DataFrame(results)
display(HTML(df_cnn_results.to_html(index=False)))

## Create Image Embeddings

#### Selected CNN

In [21]:
CNN_MODEL_FILENAME = f"{MODELS_PATH}/model_cnn.h5"
if not os.path.isfile(CNN_MODEL_FILENAME):
    default_args = {
        "kernel_size": KERNEL_SIZE,
        "strides": (1, 1),
        "pool_size": (2, 2),
        "optimizer": "Adam",
        "learning_rate": LEARNING_RATE,
        "delta": DELTA,
        "contrast_factor": CONTRAST_FACTOR,
        "is_flip_on": IS_FLIP_ON
    }
    exp_args = {
        **default_args,
        "strides": (2, 2),
        "contrast_factor": 2
    }

    # time.sleep(30)
    # clear_output(wait=True)
    print(f"Experiment: {exp_args}")

    img_seen_aug = aug_img_data(img_seen, **exp_args)
    img_seen_aug = tf.concat([img_seen, img_seen_aug], axis=0)
    y_seen_aug = tf.concat([y_seen, y_seen], axis=0)

    # shuffle the data
    reset_random_seeds()
    shuffle = tf.random.shuffle(tf.range(tf.shape(img_seen_aug)[0], dtype=tf.int32))
    img_seen_aug = tf.gather(img_seen_aug, shuffle)
    y_seen_aug = tf.gather(y_seen_aug, shuffle).numpy()

    if 'model_cnn' in locals() or 'model_cnn' in globals():
        del model_cnn
    with strategy.scope():
        model_cnn = build_cnn_model(**exp_args)
    model_cnn.fit(img_seen_aug, y_seen_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=TEST_SIZE)
    model_cnn.save(CNN_MODEL_FILENAME)

# model_cnn.summary()

In [22]:
def build_batch_embedding_df(model_embedding, data_img, index):
    # display(data_img)
    # display(index)
    embedding_img = model_embedding.predict_on_batch(data_img)
    embedding_cols = [f"emb_{i + 1}" for i in range(embedding_img.shape[1])]
    embedding_df = pd.DataFrame(embedding_img, columns=embedding_cols, index=index)
    embedding_df.reset_index(inplace=True)
    return embedding_df.rename(columns={'index': 'image'})

In [23]:
# @tf.function
# def build_embedding_df(model_embedding, dist_dataset):
#     df_embeddings_list = []
    
#     embedding_img = np.asarray([])
#     index = []
    
#     for dataset, idx in dist_dataset:
#         index.extend(idx.values)
        
#         embeddings = strategy.run(model_embedding.predict_on_batch, args=(dataset,)).values
#         embedding_img = np.append(embedding_img, embeddings, axis=0)
    
#     embedding_cols = [f"emb_{i + 1}" for i in range(embedding_img.shape[1])]
#     embedding_df = pd.DataFrame(embedding_img, columns=embedding_cols, index=index)
#     embedding_df.reset_index(inplace=True)
#     return embedding_df

In [24]:
# # get the flatten, penultimate, and output layers
# # layer_outputs = [model.get_layer('flatten'), model.layers[-3], model.layers[-1]]
# # layer_outputs = [layer.output for layer in layer_outputs]
# # model_img_embedding = keras.models.Model(inputs=model_cnn.input, outputs=layer_outputs)

# if 'model_img_embedding' in locals() or 'model_img_embedding' in globals():
#     del model_img_embedding
# # get penultimate layer
# with strategy.scope():
#     model_img_embedding = keras.models.Model(inputs=model_cnn.input, outputs=model_cnn.layers[-3].output)

# dataset_seen = tf.data.Dataset.from_tensor_slices((img_seen, df_seen['image'])).batch(GLOBAL_BATCH_SIZE) 
# dataset_test = tf.data.Dataset.from_tensor_slices((img_test, df_test['image'])).batch(GLOBAL_BATCH_SIZE) 
# dist_dataset_seen = strategy.experimental_distribute_dataset(dataset_seen)
# dist_dataset_test = strategy.experimental_distribute_dataset(dataset_test)

# df_embeddings_img_seen = build_embedding_df(model_img_embedding, dist_dataset_seen)
# display(df_embeddings_img_seen)

# df_embeddings_img_test = build_embedding_df(model_img_embedding, dist_dataset_test)
# display(df_embeddings_img_test)

In [25]:
# assert tf.distribute.get_replica_context() is not None  # default
# # dataset = tf.data.Dataset.from_tensors((list(range(4)))).batch(4)
# print(GLOBAL_BATCH_SIZE)
# dataset = tf.data.Dataset.from_tensor_slices((np.array(list(range(2 * GLOBAL_BATCH_SIZE))))).batch(GLOBAL_BATCH_SIZE)
# dist_dataset = strategy.experimental_distribute_dataset(dataset)


# tf.debugging.set_log_device_placement(True)
                                       
# with strategy.scope():
#     # assert tf.distribute.get_replica_context() is None

#     def f(i):
#         print(f"=========f: {i}")
#         replica_context = tf.distribute.get_replica_context()  # for strategy
#         assert replica_context is not None
#         tf.print("Replica id: ", replica_context.replica_id_in_sync_group + 1,
#              " of ", replica_context.num_replicas_in_sync)
#         return 2 * i

#     # print(f"ouside for: {dataset}")
#     for d in dist_dataset:
#         # print(f"inside: {d}")
#         result = strategy.run(f, args=(d,))

# tf.debugging.set_log_device_placement(False)

In [31]:
# get the flatten, penultimate, and output layers
# layer_outputs = [model.get_layer('flatten'), model.layers[-3], model.layers[-1]]
# layer_outputs = [layer.output for layer in layer_outputs]
# model_img_embedding = keras.models.Model(inputs=model_cnn.input, outputs=layer_outputs)

assert tf.distribute.get_replica_context() is not None  # default

# with strategy.scope():
    # assert tf.distribute.get_replica_context() is not None  # default
    # @tf.function


# @tf.function
def build_embedding_df(dist_dataset):
    # assert tf.distribute.get_replica_context() is None  # default

    df_embeddings_list = []

    embedding_img = np.asarray([])
    index = []
    
#     class EmbeddingModel(keras.models.Model):
#         def __init__(self):
#             super(EmbeddingModel, self).__init__()
#             model_cnn = keras.models.load_model(CNN_MODEL_FILENAME)
#             # get penultimate layer
#             self.model = keras.models.Model(inputs=model_cnn.input, outputs=model_cnn.layers[-3].output)
            
#         def call(self, inputs):
#             return self.model.predict(inputs)

    @tf.function
    def build_embedding_in_batch(dataset):
        # print(dataset.shape)
        # return tf.distribute.get_replica_context().all_gather(model_embedding.predict(dataset), axis=0)
        return tf.distribute.get_replica_context().all_gather(model_embedding(dataset), axis=0)
    
    @tf.function
    def return_const(idx):
        return idx

    embedding_img = np.empty((0, 256), np.float32)
    index = np.empty((0), np.int32)
    with strategy.scope():
        model_cnn = keras.models.load_model(CNN_MODEL_FILENAME)
        model_embedding = keras.models.Model(inputs=model_cnn.input, outputs=model_cnn.layers[-3].output)

        for dataset, idx in dist_dataset:
            # per_replica_idx = strategy.run(return_const, args=(idx,))
            for ix in strategy.experimental_local_results(idx):
                print(len(ix))
                index = np.append(index, ix.numpy(), axis=0)
            print(f"index: {index.shape}")
            
            for ix in strategy.experimental_local_results(dataset):
                print(f"dataset-shape: {len(ix)}")
                
            per_replica_embedding = strategy.run(build_embedding_in_batch, args=(dataset,))
            # per_replica_embedding = build_embedding_in_batch(dataset)
            # per_replica_embedding = tf.distribute.get_replica_context().all_gather(model_embedding(dataset), axis=0)
            for embedding in strategy.experimental_local_results(per_replica_embedding):
                embedding_img = np.append(embedding_img, embedding.numpy(), axis=0)
            print(f"embedding_img: {embedding_img.shape}")
        
        del model_embedding
        del model_cnn

    # print(index)
    print(index.shape)
    print(embedding_img.shape)
    embedding_cols = [f"emb_{i + 1}" for i in range(embedding_img.shape[1])]
    embedding_df = pd.DataFrame(embedding_img, columns=embedding_cols, index=index)
    embedding_df.reset_index(inplace=True)
    return embedding_df

# with strategy.scope():
if 'model_img_embedding' in locals() or 'model_img_embedding' in globals():
    del model_img_embedding
if 'model_cnn' in locals() or 'model_cnn' in globals():
    del model_cnn
# model_cnn = keras.models.load_model(CNN_MODEL_FILENAME)
# # get penultimate layer
# model_embedding = keras.models.Model(inputs=model_cnn.input, outputs=model_cnn.layers[-3].output)

dataset_seen = tf.data.Dataset.from_tensor_slices((img_seen, df_seen['image'].to_list())).batch(GLOBAL_BATCH_SIZE) 
dist_dataset_seen = strategy.experimental_distribute_dataset(dataset_seen)
df_embeddings_img_seen = build_embedding_df(dist_dataset_seen)
display(df_embeddings_img_seen)

dataset_test = tf.data.Dataset.from_tensors((img_test, df_test['image'].to_list())).batch(GLOBAL_BATCH_SIZE) 
dist_dataset_test = strategy.experimental_distribute_dataset(dataset_test)
df_embeddings_img_test = build_embedding_df(dist_dataset_test)
display(df_embeddings_img_test)

2022-11-26 09:26:00.378977: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 128
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:85"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 224
        }
        dim {
          size: 224
        }
        dim {
          size: 3
        }
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  ar

32
32
index: (64,)
dataset-shape: 32
dataset-shape: 32


AttributeError: in user code:

    File "/tmp/ipykernel_667086/1254733102.py", line 36, in build_embedding_in_batch  *
        return tf.distribute.get_replica_context().all_gather(model_embedding(dataset), axis=0)

    AttributeError: 'NoneType' object has no attribute 'all_gather'


In [ ]:
df_complete_seen = df_seen.join(df_embeddings_img_seen.set_index('image'), on='image', how='left')
display(df_complete_seen)

df_complete_test = df_test.join(df_embeddings_img_test.set_index('image'), on='image', how='left')
display(df_complete_test)

## Train-Val-Test split

In [ ]:
columns_cat = ['anatom_site_general', 'sex', 'anatomy_site']
columns_to_drop = [*columns_cat, 'image', 'lesion_id', 'label']


def usable_df(df, columns=None):
    df_dump = df.copy()
    y = df_dump['label']

    for cat in columns_cat:
        one_hot = pd.get_dummies(df_dump[cat], prefix=cat)
        df_dump = df_dump.join(one_hot)

    df_dump = df_dump.drop(columns_to_drop, axis=1)
    if columns is not None:
        missing_cols = set(columns).difference(set(df_dump.columns))
        for miss_col in missing_cols:
            df_dump[miss_col] = np.zeros(df_dump.shape[0])
        df_dump = df_dump[columns]
    return df_dump, df_dump.to_numpy(), y, df_dump.columns


_, X_seen, y_seen, cols_usable = usable_df(df_complete_seen)
X_train, X_val, y_train, y_val = train_test_split(X_seen, y_seen, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print(f"Length of training data: {X_train.shape}")
print(f"Length of validation data: {X_val.shape}")

_, X_test, y_test, _ = usable_df(df_complete_test, cols_usable)
print(f"Length of test data: {X_test.shape}")

In [ ]:
def display_results(model):
    for name, X, y_true in [
        ("Train", X_train, y_train),
        ("Validation", X_val, y_val),
        ("Test", X_test, y_test)
    ]:
        print(f"=============== {name} dataset ===============")
        y_pred = model.predict(X)
        print(classification_report(y_true, y_pred))
    sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d')
    plt.show()

## Logistic Regression

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# from the above link, some note-worthy points are below:
# * For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones;
# * For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss;
# So, only "newton-cg", "sag", "saga", "lbfgs" are used and max_iter is set to 1000 to overcome ConvergenceWarning

from sklearn.linear_model import LogisticRegression

for solver in ["newton-cg", "sag", "saga", "lbfgs"]:
    print(f"Solver: {solver}")
    classifier_logistic = LogisticRegression(random_state=RANDOM_STATE, solver=solver, max_iter=1000)
    classifier_logistic.fit(X_train, y_train)
    display_results(classifier_logistic)

## Naive Bayes

In [ ]:
# https://scikit-learn.org/stable/modules/naive_bayes.html

from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
# CategoricalNB is suitable only for classification with discrete features that are categorically distributed, which is not suitable for the data available and so is skipped.

for naive_bayes in [GaussianNB, MultinomialNB, ComplementNB, BernoulliNB]:
    classifier_nb = naive_bayes()
    classifier_nb.fit(X_train, y_train)
    display_results(classifier_nb)

## KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn = KNeighborsClassifier()
classifier_knn.fit(X_train, y_train)
display_results(classifier_knn)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier_forest = RandomForestClassifier(max_depth=32, random_state=RANDOM_STATE)
classifier_forest.fit(X_train, y_train)
display_results(classifier_forest)

## Adaboost Trees

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

classifier_adaboost = AdaBoostClassifier(random_state=RANDOM_STATE)
classifier_adaboost.fit(X_train, y_train)
display_results(classifier_adaboost)

## SVM

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

classifier_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
classifier_svm.fit(X_train, y_train)
display_results(classifier_svm)

## RNN

## LSTM

In [ ]:
def build_model_lstm():
    clear_session()
    reset_random_seeds()

    model = keras.Sequential()
    model.add(keras.layers.LSTM(128))
    model.add(keras.layers.Dense(units=len(labels), name='output', activation="softmax"))

    model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model


model_lstm = build_model_lstm()
model_lstm.build(input_shape=(None, 1, X_seen.shape[1]))
model_lstm.summary()

history_lstm = model_lstm.fit(np.reshape(X_seen, (X_seen.shape[0], 1, X_seen.shape[1])), y_seen, epochs=EPOCHS,
                              batch_size=BATCH_SIZE, validation_split=TEST_SIZE).history
plot_history(history_lstm)

print(f"Test Accuracy: {model_lstm.evaluate(X_test, y_test)[1] * 100:.2f}")

## GRUs